In [2]:
import os
from resnet_model import *
from mcts import *
from distance import *
from blockMeshDict_v3 import *
from input_data import *


n = 21
num = n
c_n2 = n * (n - 1) // 2

pi = np.zeros([n - 2, n * (n - 1) // 2])
air_input, _ = inputs_data(naca0012(n))

airfoil = np.zeros([n, n])
airfoil[n // 2, [0, n - 1]] = 1


x = tf.placeholder(tf.float32)

pre, pro, val = model(inputs=x, filters=32, blocks=8, n=num, training=False)
saver = tf.train.Saver()

z = tf.placeholder(dtype=tf.float32, shape=[None, 1])
pi_true = tf.placeholder(dtype=tf.float32, shape=[None, c_n2])

train, loss = mcts_train(z, val, pi_true, pre)


if __name__ == '__main__':

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    saver.restore(sess, '/home/xie/tf_model/x')

    for _ in range(1):
        wing = Wing(airfoil)
        for i in range(n - 2):
            print(i)
            pi[i], point = uct(wing, n * 10, pro, val, x, sess)
            print(pi[i])
            wing.draw(point)

        vertices = cal_vertices(wing.airfoil)
        write_dict(vertices)
        print(wing.airfoil)

        if _ > 0:
            os.system('./wing_openFoam/Allclean')
        os.system('blockMesh -case "wing_openFoam" > blockMesh.log')
        os.system('rhoSimpleFoam -case "wing_openFoam" > rhoSimple.log')
        os.system('paraFoam -touch -case "wing_openFoam"')
        os.system('mv wing_openFoam/wing_openFoam.OpenFOAM wing_openFoam/wing_openFoam.foam')
        os.system('pvpython wing_openFoam/scipt.py')

        D = np.zeros(n - 2).reshape([-1, 1])
        dis = 2 * distance() - 1
        print(dis)
        D += dis

        for i in range(3000):
            _, loss_value = sess.run((train, loss), {x: air_input, z: D, pi_true: pi})
            if i % 100 == 0:
                print(loss_value)

        print(wing.airfoil)


INFO:tensorflow:Restoring parameters from /home/xie/tf_model/x


NotFoundError: Key batch_normalization_28/beta not found in checkpoint
	 [[Node: save_1/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_1/Const_0_0, save_1/RestoreV2/tensor_names, save_1/RestoreV2/shape_and_slices)]]
	 [[Node: save_1/RestoreV2/_225 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_230_save_1/RestoreV2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'save_1/RestoreV2', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/xie/tensorflow/lib/python3.6/site-packages/ipyker